In [64]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data_url = 'loan_applic.csv'  # Replace this with your dataset URL or local path
df = pd.read_csv(data_url)

# Split the data into features and target variable
X = df.drop(columns=['Application Number', 'Outcome',

                    ])  # Exclude non-predictive columns
X = X.replace(',', '', regex=True)
#X = X.fillna('Missing')
# Encode categorical labels
label_encoder = LabelEncoder()
df['Outcome'] = label_encoder.fit_transform(df['Outcome'])
y = df['Outcome']  # Assuming 'Outcome' is the target variable for regression

# Train the model
cat_features = [
                     'Application: Remittance Frequency',
                     'Primary Contact Gender',
                     'Application: Close Date',
                     'crime_record',
                     'Has Website',
                     'Industry',
                     'Office Space',
                     'Position',
                     'Shipping State',
                     'Type',
]
# Replace NaN values with 'Missing' in specified columns
for col in cat_features:
    X[col] = X[col].fillna('Missing')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=1000,  # Number of trees (boosting iterations)
                          learning_rate=0.1,  # Step size shrinkage used in update to prevent overfitting
                          depth=6,  # Depth of trees
                          loss_function='RMSE',  # Loss function for regression
                          random_state=42)



model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100,
          cat_features=cat_features
         )

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

0:	learn: 0.4412956	test: 0.4396796	best: 0.4396796 (0)	total: 9.44ms	remaining: 9.43s
100:	learn: 0.3971960	test: 0.4219656	best: 0.4219124 (93)	total: 781ms	remaining: 6.95s
200:	learn: 0.3758538	test: 0.4213942	best: 0.4209813 (151)	total: 1.53s	remaining: 6.08s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.420981329
bestIteration = 151

Shrink model to first 152 iterations.
Mean Squared Error: 0.17722527935623264
